In [1]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import joblib

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold,KFold
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score
import os

from datasets import concatenate_datasets,load_dataset,load_from_disk
from transformers import AutoModel, AutoTokenizer, AdamW, DataCollatorWithPadding

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pytl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import polars as pl
from torch_geometric.nn import GCNConv,SAGEConv


In [3]:
with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    pid = json.load(file)
train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

piddf = joblib.load('data/pid_df.pkl')

data = pd.concat([train,valid,test]).reset_index(drop = True)
piddf = piddf[piddf['id'].isin(set(data['PID']))]

In [5]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def l2_normalize(vector):
    norm = np.linalg.norm(vector, ord=2)
    if norm == 0:
        return vector
    return vector / norm

bge_m3_tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')
bge_m3_model = AutoModel.from_pretrained('BAAI/bge-m3')

device = "cuda:0" if torch.cuda.is_available() else "cpu"




In [7]:
def bge_m3_text2vec(text):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    encoded_input = bge_m3_tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    bge_m3_model_input = bge_m3_model.to(device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = bge_m3_model_input(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Convert to a list and return as JSON response
    embeddings_list = sentence_embeddings.tolist()
    # normalize handle
    normalized_embeddings_list = l2_normalize(np.array(embeddings_list))
    normalized_embeddings_list.tolist()
    return normalized_embeddings_list.tolist()

In [8]:
piddf['title'] = piddf['title'].apply(lambda x:x.lower())
piddf = piddf.reset_index(drop = True)

title_emb = []
batch_size = 32
for i in tqdm(range(piddf.shape[0]//batch_size + 1)):
    text = piddf[i*32:(i+1)*32]['title'].to_list()
    title_emb.append(bge_m3_text2vec(text))



100%|██████████| 93/93 [00:10<00:00,  9.07it/s]


In [9]:
title_emb = np.concatenate(title_emb)
t_dict = dict(zip(piddf['id'].to_list(),title_emb))
joblib.dump(t_dict,'bge_m3_emb/title2vec_dict.pkl')

['bge_m3_emb/title2vec_dict.pkl']

In [10]:

def cos_similarity(target, embedding):
    numerator = np.sum(target * embedding, axis=1)
    denominator = np.sqrt(np.sum(np.square(target)) * np.sum(np.square(embedding),axis=1))
    return numerator / denominator

ans = []
for autherID in tqdm(data['autherID'].unique()):
    pid_list = data[data['autherID'] == autherID]['PID'].to_list()
    for f1 in pid_list:
        x1 = t_dict.get(f1)
        x2 = [f for f in pid_list if f != f1]
        x2 = [t_dict.get(f) for f in x2]
        ans.append(list(cos_similarity(x1, x2)))
data['pid_title_sim'] = ans

for f in ['pid_title_sim']:
    data[f + '_mean'] = data[f].apply(lambda x:np.mean(x))
    data[f + '_max'] = data[f].apply(lambda x:np.max(x))
    data[f + '_min'] = data[f].apply(lambda x:np.min(x))
    data[f + '_std'] = data[f].apply(lambda x:np.std(x))
    data[f + '_median'] = data[f].apply(lambda x:np.median(x))
    
data = data.drop(['pid_title_sim'],axis = 1)

100%|██████████| 15/15 [00:05<00:00,  2.60it/s]


In [11]:
data[['PID', 'autherID','pid_title_sim_mean',
       'pid_title_sim_max', 'pid_title_sim_min', 'pid_title_sim_std',
       'pid_title_sim_median']].to_feather('bge_m3_emb_title_feat.pkl')

In [12]:
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs
def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.replace('\r\n',' [SEP]').replace('\n',' [BR]').replace("\'","'")
        .encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

def resolve_encodings_and_normalize(text: str) -> str:
    try:
        """Resolve the encoding problems and normalize the abnormal characters."""
        text = (
            text.replace('\n',' ').replace("\'","'")
            .encode("raw_unicode_escape")
            .decode("utf-8", errors="replace_decoding_with_cp1252")
            .encode("cp1252", errors="replace_encoding_with_utf8")
            .decode("utf-8", errors="replace_decoding_with_cp1252")
        )
        text = unidecode(text)
        return text
    except:
        return ''


In [13]:
piddf['abstract'] = piddf['abstract'].apply(lambda x:x.lower())
piddf.loc[i*4+2,'abstract'] = ''
piddf['abstract'] = piddf['abstract'].map(resolve_encodings_and_normalize)
piddf['abstract'] = piddf['abstract'].apply(lambda x:x[:2048])
piddf = piddf.reset_index(drop = True)
abstract_emb = []
batch_size = 4
for i in tqdm(range(piddf.shape[0]//batch_size + 1)):
    text = piddf[i*batch_size:(i+1)*batch_size]['abstract'].to_list()
    abstract_emb.append(bge_m3_text2vec(text))
    


100%|██████████| 740/740 [00:44<00:00, 16.74it/s]


In [14]:
abstract_emb = np.concatenate(abstract_emb)
t_dict = dict(zip(piddf['id'].to_list(),abstract_emb))
joblib.dump(t_dict,'bge_m3_emb/abstract2vec_dict.pkl')

['bge_m3_emb/abstract2vec_dict.pkl']

In [15]:

def cos_similarity(target, embedding):
    numerator = np.sum(target * embedding, axis=1)
    denominator = np.sqrt(np.sum(np.square(target)) * np.sum(np.square(embedding),axis=1))
    return numerator / denominator

ans = []
for autherID in tqdm(data['autherID'].unique()):
    pid_list = data[data['autherID'] == autherID]['PID'].to_list()
    for f1 in pid_list:
        x1 = t_dict.get(f1)
        x2 = [f for f in pid_list if f != f1]
        x2 = [t_dict.get(f) for f in x2]
        ans.append(list(cos_similarity(x1, x2)))
data['pid_abstract_sim'] = ans



100%|██████████| 15/15 [00:06<00:00,  2.50it/s]


In [16]:
for f in ['pid_abstract_sim']:
    data[f + '_mean'] = data[f].apply(lambda x:np.mean(x))
    data[f + '_max'] = data[f].apply(lambda x:np.max(x))
    data[f + '_min'] = data[f].apply(lambda x:np.min(x))
    data[f + '_std'] = data[f].apply(lambda x:np.std(x))
    data[f + '_median'] = data[f].apply(lambda x:np.median(x))

#data = data.drop(['pid_title_sim'],axis = 1)
data = data.drop(['autherName'],axis = 1)


In [17]:
data[['PID', 'autherID','pid_abstract_sim_mean',
       'pid_abstract_sim_max', 'pid_abstract_sim_min', 'pid_abstract_sim_std',
       'pid_abstract_sim_median']].to_feather('feats/bge_m3_abstract_sim_feats.feather')